Задание 5. Сравнить две модели трансформеров на примере машинного перевода. 

Перевод следующий: русский -> английский -> испанский -> арабский -> русский. Сделать выводы.

In [1]:
from transformers import MarianMTModel, MarianTokenizer
import torch

def translate_helsinki(text, model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    translated = model.generate(**inputs)

    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated][0]

text = "Допустим, я тут что-то должен написать оригинальное. Я люблю хинкали и не только"

translated_helsinki = translate_helsinki(text, "Helsinki-NLP/opus-mt-ru-en")
translated_helsinki = translate_helsinki(translated_helsinki, "Helsinki-NLP/opus-mt-en-es")
translated_helsinki = translate_helsinki(translated_helsinki, "Helsinki-NLP/opus-mt-es-ar")
final_helsinki = translate_helsinki(translated_helsinki, "Helsinki-NLP/opus-mt-ar-ru")

print("Оригинальный текст:", text)
print("Перевод через Helsinki-NLP:", final_helsinki)


Оригинальный текст: Допустим, я тут что-то должен написать оригинальное. Я люблю хинкали и не только
Перевод через Helsinki-NLP: Скажем так, я должен написать что-то оригинальное.


In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

def translate(text, model_name, src_lang, tgt_lang, device):
    tokenizer = M2M100Tokenizer.from_pretrained(model_name)
    model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)

    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

text = "Допустим, я тут что-то должен написать оригинальное. Я люблю хинкали и не только"
model_name_facebook = "facebook/m2m100_418M"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

translated_facebook = translate(text, model_name_facebook, "ru", "en", device)
translated_facebook = translate(translated_facebook, model_name_facebook, "en", "es", device)
translated_facebook = translate(translated_facebook, model_name_facebook, "es", "ar", device)
translated_facebook = translate(translated_facebook, model_name_facebook, "ar", "ru", device)

print("Оригинальный текст:", text)
print("Перевод с использованием Facebook:", translated_facebook)


Оригинальный текст: Допустим, я тут что-то должен написать оригинальное. Я люблю хинкали и не только
Перевод с использованием Facebook: Мы признаем, что мне нужно что-то написать здесь.Мне нравятся углы, а не только


Вывод: обе модели справились не очень, но шизофазия от фейсбука мне понравилась больше